<a href="https://colab.research.google.com/github/mrishikreddy/nlp/blob/main/Assignment8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [4]:
# Step 1: Prepare the Data
text = """Once upon a time, there was a little girl named Red Riding Hood.
          She loved to visit her grandmother, who lived in the woods.
          One day, her mother asked her to take a basket of goodies to her grandmother.
          On her way through the woods, she met a big bad wolf who wanted to eat her."""

# Tokenize and sequence data
tokenizer = Tokenizer(char_level=True)  # Using character-level tokenization
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for i in range(1, len(text)):
    n_gram_sequence = text[:i+1]
    token_list = tokenizer.texts_to_sequences([n_gram_sequence])[0]
    input_sequences.append(token_list)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Separate inputs and labels
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Step 2: Define the Transformer Model
def build_transformer_model(vocab_size, seq_len, embed_dim=32, num_heads=2, ff_dim=32):
    inputs = tf.keras.layers.Input(shape=(seq_len,))
    embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_dim)(inputs)
    transformer_block = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(embedding_layer, embedding_layer)
    transformer_block = tf.keras.layers.GlobalAveragePooling1D()(transformer_block)  # Reduce sequence dimension
    ff_layer = tf.keras.layers.Dense(ff_dim, activation='relu')(transformer_block)
    outputs = tf.keras.layers.Dense(vocab_size, activation='softmax')(ff_layer)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

model = build_transformer_model(total_words, max_sequence_len - 1)

# Step 3: Compile and Train the Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train for different epochs and compare results
for epochs in [20, 60, 70]:
    model.fit(X, y, epochs=epochs, verbose=1)
    model.save(f"transformer_model_{epochs}_epochs.h5")

# Step 4: Generate Text
def generate_text(seed_text, next_words=50, max_sequence_len=max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += output_word
    return seed_text

seed_text = "Once upon a time, there was a little girl named Red Riding Hood."
generated_text = generate_text(seed_text)
print(generated_text)

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 199ms/step - accuracy: 0.1308 - loss: 3.2836
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 213ms/step - accuracy: 0.2730 - loss: 3.1582
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 139ms/step - accuracy: 0.2673 - loss: 2.9341
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 150ms/step - accuracy: 0.2883 - loss: 2.7704
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 149ms/step - accuracy: 0.2873 - loss: 2.6656
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.2671 - loss: 2.7691
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 206ms/step - accuracy: 0.3086 - loss: 2.6146
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 174ms/step - accuracy: 0.2737 - loss: 2.7310
Epoch 9/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 139ms/step - accuracy: 0.2441 - loss: 2.7276
Epoch 10/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 152ms/step - accuracy: 0.2537 - loss: 2.7057
Epoch 11/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 147ms/step - accuracy: 0.2779 - loss: 2.6831
Epoch 12/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 146ms/step

Epoch 1/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 149ms/step - accuracy: 0.2929 - loss: 2.6409
Epoch 2/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 149ms/step - accuracy: 0.2911 - loss: 2.6165
Epoch 3/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.3056 - loss: 2.6120
Epoch 4/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 180ms/step - accuracy: 0.3145 - loss: 2.5884
Epoch 5/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 161ms/step - accuracy: 0.2595 - loss: 2.6766
Epoch 6/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 137ms/step - accuracy: 0.2640 - loss: 2.6219
Epoch 7/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 151ms/step - accuracy: 0.2977 - loss: 2.5643
Epoch 8/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 153ms/step - accuracy: 0.2841 - loss: 2.6185
Epoch 9/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step - accuracy: 0.3169 - loss: 2.5450
Epoch 10/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 284ms/step - accuracy: 0.2696 - loss: 2.6565
Epoch 11/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 163ms/step - accuracy: 0.2699 - loss: 2.6391
Epoch 12/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 151ms/step

Epoch 1/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 268ms/step - accuracy: 0.2911 - loss: 2.5262
Epoch 2/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 143ms/step - accuracy: 0.2767 - loss: 2.5626
Epoch 3/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 154ms/step - accuracy: 0.2900 - loss: 2.5742
Epoch 4/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 141ms/step - accuracy: 0.2886 - loss: 2.5615
Epoch 5/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 152ms/step - accuracy: 0.2852 - loss: 2.6091
Epoch 6/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 242ms/step - accuracy: 0.3132 - loss: 2.4961
Epoch 7/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 142ms/step - accuracy: 0.2631 - loss: 2.5485
Epoch 8/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 144ms/step - accuracy: 0.2599 - loss: 2.5685
Epoch 9/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - accuracy: 0.2995 - loss: 2.5348
Epoch 10/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 142ms/step - accuracy: 0.2965 - loss: 2.5188
Epoch 11/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 146ms/step - accuracy: 0.2324 - loss: 2.6305
Epoch 12/70
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 207ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━